In [1]:
import pandas as pd

In [16]:
data = pd.read_excel('bpi_challenge_2018.xlsx')

In [ ]:
# drop uncomplete cases (2017 mostly) and thus do not have a complete label

## Undesired Outcome 1

Undesired outcome 1: The payment is late. A payment can be considered
timely, if there has been a begin payment activity by the end of the year that was
not eventually followed by abort payment.

Late Payments There is no easy way of filtering the cases to find cases with
late payments. So in order to look at the differences, we added a case attribute
ourselves with the use of Python. With the use of the ProM forum, we identified
3 situations in which a case is considered late:
1. There is no ‘begin payment’ event in the case.
2. The last ‘begin payment’ event is followed by an ‘abort payment’ event.
3. The last ‘begin payment’ event occurs in a later year than what the case was
started in.

In [ ]:
# group by per caseid --> to find latest payment

data.loc[.groupby('id').date.idxmax()]


In [17]:
#1 group by per caseid --> to find latest payment per Case

df = data.loc[data['Activity'] == 'Payment application-Application-begin payment']
df = df.rename(columns = {'Complete Timestamp': 'Timestamp'})
latest_payments =df.loc[df.groupby('Case ID').Timestamp.idxmax()]
for i in latest_payments.index:
    # choose which dataframe to update
    # for now pick dataframe: data
    
    data.loc[i,'Latest_Begin_Payment'] = 1

In [18]:
#2 Find latest abort payment per Case
df = data.loc[data['Activity'] == 'Payment application-Application-abort payment']
df = df.rename(columns = {'Complete Timestamp': 'Timestamp'})
latest_abort =df.loc[df.groupby('Case ID').Timestamp.idxmax()]
for i in latest_abort.index:
    # choose which dataframe to update
    # for now pick dataframe: data
    
    data.loc[i,'Latest_Abort_Payment'] = 1

In [19]:
#2 Find either of the two activities
df = data.loc[(data['Latest_Begin_Payment'] == 1) | (data['Latest_Abort_Payment'] ==1)]
df = df.rename(columns = {'Complete Timestamp': 'Timestamp'})
abort_or_paid = df.loc[df.groupby('Case ID').Timestamp.idxmax()]
for i in abort_or_paid.index:
    if data.loc[i, 'Latest_Begin_Payment'] == 1:
        data.loc[i, 'Aborted'] = 1
    else:
        data.loc[i, 'Not_aborted'] = 1
data.head()


,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,...,doctype,eventid,lifecycle:transition,note,subprocess,success,Latest_Begin_Payment,Latest_Abort_Payment,Aborted,Not_aborted
0,788e3f355afbed07,Payment application-Application-mail income,0;n/a,2015-04-16,Variant 6000,6000,401.63,NaN,NaN,NaN,...,Payment application,NaN,complete,none,Application,True,NaN,NaN,NaN,NaN
1,788e3f355afbed07,Entitlement application-Main-mail valid,0;n/a,2015-04-27,Variant 6000,6000,401.63,NaN,NaN,NaN,...,Entitlement application,NaN,complete,none,Main,True,NaN,NaN,NaN,NaN
2,788e3f355afbed07,Payment application-Application-mail valid,0;n/a,2015-04-27,Variant 6000,6000,401.63,NaN,NaN,NaN,...,Payment application,NaN,complete,none,Application,True,NaN,NaN,NaN,NaN
3,788e3f355afbed07,Entitlement application-Main-mail valid,0;n/a,2015-04-27,Variant 6000,6000,401.63,NaN,NaN,NaN,...,Entitlement application,NaN,complete,none,Main,True,NaN,NaN,NaN,NaN
4,788e3f355afbed07,Parcel document-Main-initialize,Document processing automaton,2015-06-15,Variant 6000,6000,401.63,NaN,NaN,NaN,...,Parcel document,-1.800866e+16,complete,none,Main,True,NaN,NaN,NaN,NaN


In [25]:
#3 find whether first occurence is in the year before final last payment
#identify first event
df = data
df = df.rename(columns = {'Complete Timestamp': 'Timestamp'})
first_event = df.loc[df.groupby('Case ID').Timestamp.idxmin()]
for i in first_event.index:
    data.loc[i, 'first_event'] = 1

In [38]:
data['Complete Timestamp'][0].year

2015

In [41]:
df = data.loc[(data['Latest_Begin_Payment'] == 1) | (data['first_event'] ==1)]
df = df.rename(columns = {'Complete Timestamp':'Timestamp'})
for i in df.index:
    print(i)
#     if data.loc[i, 'Case ID']
# df_in['value'] = (df_in[['A', 'B']] != df_in[['A', 'B']].shift()).any(axis=1).cumsum()

0
43
45
90
96
157
165
208
210
267
269
357
359
434
436
475
478
524
526
566
573
613
615
663
665
712
715
759
765
808
811
864
920
967
971
1015
1018
1072
1074
1119
1122
1169
1171
1224
1226
1435
1477
1517
1524
1573
1575
1620
1654
1694
1696
1737
1739
1790
1800
1853
1875
1923
1927
1972
1983
2031
2033
2083
2085
2133
2135
2179
2201
2249
2251
2302
2306
2375
2388
2431
2433
2522
2524
2568
2576
2629
2640
2693
2702
2744
2746
2794
2801
2848
2876
2916
2920
2978
3047
3152
3154
3201
3203
3255
3261
3319
3326
3379
3381
3431
3433
3476
3493
3558
3560
3601
3603
3647
3650
3692
3717
3780
3786
3832
3834
3888
3900
3968
3970
4101
4105
4150
4154
4197
4204
4251
4253
4305
4307
4354
4389
4436
4442
4489
4491
4536
4538
4579
4582
4626
4629
4687
4712
4763
4767
4825
4827
4864
4866
4910
4912
4974
5005
5058
5112
5163
5228
5272
5288
5332
5362
5405
5413
5455
5457
5646
5650
5698
5700
5747
5792
5852
5858
5904
5908
5969
5971
6013
6015
6062
6068
6116
6134
6177
6181
6237
6239
6289
6291
6338
6340
6384
6392
6450
6452
6510
6513
6572
6

In [ ]:
# Condition 1: filter out cases with late payment

## Undesired Outcome 2